# Blending problem

Problem summary: Find the cheapest combination of ingredients that meet nutritional requirements 

Problem details: [see here](https://pythonhosted.org/PuLP/CaseStudies/a_blending_problem.html)

The purpose of this notebook is to try out Google's [OR-Tools](https://developers.google.com/optimization).

The exercise below was taken from [here](https://pythonhosted.org/PuLP/CaseStudies/a_blending_problem.html).

In [1]:
from ortools.linear_solver import pywraplp
import pandas as pd
from loguru import logger

## 1. Define data

In [2]:
def create_data_model():
    config = {}
    # unit: x per gram
    config['food_data_col'] = ['cost', 'ingredient', 'protein', 'fat', 'fibre', 'salt']

    config['food_data'] = [
        ['chicken', 0.013, 0.100, 0.080, 0.001, 0.002],
        ['beef', 0.008, 0.200, 0.100, 0.005, 0.005],
        ['mutton', 0.010, 0.150, 0.110, 0.003, 0.007],
        ['rice', 0.002, 0.000, 0.010, 0.100, 0.002],
        ['wheat', 0.005, 0.040, 0.010, 0.150, 0.008],
        ['gel', 0.001, 0.000, 0.000, 0.000, 0.000],
    ]

    # nutrition per gram
    config['nutrition_req'] = [
        ['protein', '>=', 0.08],
        ['fat', '>=', 0.06],
        ['fibre', '<=', 0.02],
        ['salt', '<=', 0.004],
    ]

    # weight in gram
    config['weight_per_can'] = 100
    
    return config

In [3]:
config = create_data_model()

## 2. Declare solver

GLOP is Google's linear programming system.

In [4]:
# Instantiate a Glop solver, naming it SolveBlending.
solver = pywraplp.Solver('SolveBlending',
                       pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

# number of milliseconds
solver.SetTimeLimit(60*100)

## 3. Declare decision variables

In [5]:
amount = [[]] * len(config['food_data'])

for i in range(0, len(config['food_data'])):
    # Declare continuous variables (lb, ub, name)
    amount[i] = solver.NumVar(0.0, solver.infinity(), config['food_data'][i][0])
    logger.debug(f'amount[{i}] = Amount in grams of {amount[i]} in a can of cat food')

2020-02-12 22:28:20.526 | DEBUG    | __main__:<module>:6 - amount[0] = Amount in grams of chicken in a can of cat food
2020-02-12 22:28:20.529 | DEBUG    | __main__:<module>:6 - amount[1] = Amount in grams of beef in a can of cat food
2020-02-12 22:28:20.534 | DEBUG    | __main__:<module>:6 - amount[2] = Amount in grams of mutton in a can of cat food
2020-02-12 22:28:20.539 | DEBUG    | __main__:<module>:6 - amount[3] = Amount in grams of rice in a can of cat food
2020-02-12 22:28:20.543 | DEBUG    | __main__:<module>:6 - amount[4] = Amount in grams of wheat in a can of cat food
2020-02-12 22:28:20.549 | DEBUG    | __main__:<module>:6 - amount[5] = Amount in grams of gel in a can of cat food


## 3. Set objective: minimize the cost of a can of cat food

Objective: $min (\$0.013Amount_{chicken} + \$0.008Amount_{beef} + \$0.010Amount_{mutton} + \$0.002Amount_{rice} + \$0.005Amount_{wheat} + \$0.001Amount_{gel}) $

In [9]:
objective = solver.Objective()
for i in range(0, len(config['food_data'])):
    # Set coefficient of each variable in the objective
    # There is no coefficient for any of the terms, so set to 1
    objective.SetCoefficient(amount[i], config['food_data'][i][1])
    logger.debug(f"{config['food_data'][i][1]:.3f} * {amount[i]}")

# declares to be a minimization problem
objective.SetMinimization()

2020-02-12 22:29:18.557 | DEBUG    | __main__:<module>:6 - 0.013 * chicken
2020-02-12 22:29:18.559 | DEBUG    | __main__:<module>:6 - 0.008 * beef
2020-02-12 22:29:18.563 | DEBUG    | __main__:<module>:6 - 0.010 * mutton
2020-02-12 22:29:18.566 | DEBUG    | __main__:<module>:6 - 0.002 * rice
2020-02-12 22:29:18.567 | DEBUG    | __main__:<module>:6 - 0.005 * wheat
2020-02-12 22:29:18.567 | DEBUG    | __main__:<module>:6 - 0.001 * gel


## 4. Create constraints

1. The total amount in grams must sum to 1

2. For each gram of food, 
  1. protein >= 0.08g
  2. fat >= 0.06g
  3. fibre <= 0.02g
  4. salt <= 0.004g


In [10]:
def print_constraint(idx, constraint):
    logger.debug(f'constraint {idx}: lb {constraint.lb()}; ub {constraint.ub()}; \
                   coefficients {list(zip([constraint.GetCoefficient(a) for a in amount], amount))}')

total_constraints = 1 + len(config['nutrition_req'])
constraints = [0] * total_constraints
constraint_pointer = 0

# want to find out the combination of ingredients per gram
# as everything is defined per gram
constraints[0] = solver.Constraint(1, 1)
for amt in amount:
    constraints[0].SetCoefficient(amt, 1)
print_constraint(constraint_pointer, constraints[constraint_pointer])
constraint_pointer += 1


# nutrition requirements
for i, req in enumerate(config['nutrition_req']):
    if req[1] == '>=':
        constraints[constraint_pointer] = solver.Constraint(req[2], solver.infinity())
    elif req[1] == '<=':
        constraints[constraint_pointer] = solver.Constraint(0, req[2])
        
    for j, ingredient in enumerate(config['food_data']):
        constraints[constraint_pointer].SetCoefficient(amount[j], ingredient[i+2])
    
    print_constraint(constraint_pointer, constraints[constraint_pointer])
    constraint_pointer += 1

2020-02-12 22:29:25.582 | DEBUG    | __main__:print_constraint:3 - constraint 0: lb 1.0; ub 1.0;                    coefficients [(1.0, chicken), (1.0, beef), (1.0, mutton), (1.0, rice), (1.0, wheat), (1.0, gel)]
2020-02-12 22:29:25.583 | DEBUG    | __main__:print_constraint:3 - constraint 1: lb 0.08; ub inf;                    coefficients [(0.1, chicken), (0.2, beef), (0.15, mutton), (0.0, rice), (0.04, wheat), (0.0, gel)]
2020-02-12 22:29:25.585 | DEBUG    | __main__:print_constraint:3 - constraint 2: lb 0.06; ub inf;                    coefficients [(0.08, chicken), (0.1, beef), (0.11, mutton), (0.01, rice), (0.01, wheat), (0.0, gel)]
2020-02-12 22:29:25.588 | DEBUG    | __main__:print_constraint:3 - constraint 3: lb 0.0; ub 0.02;                    coefficients [(0.001, chicken), (0.005, beef), (0.003, mutton), (0.1, rice), (0.15, wheat), (0.0, gel)]
2020-02-12 22:29:25.589 | DEBUG    | __main__:print_constraint:3 - constraint 4: lb 0.0; ub 0.004;                    coefficients [

## 5. Solve

In [11]:
status = solver.Solve()

## 6. Display solution

In [21]:
if status == solver.OPTIMAL:
    # Display the amounts (in dollars) to purchase of each food.
    price_total = 0
    amount_total = 0
    num_nutrients = len(config['nutrition_req'])
    nutrients = [0] * num_nutrients
    
    for i in range(0, len(amount)):
        amount_total += amount[i].solution_value()
        price_total += amount[i].solution_value() * config['food_data'][i][1]

        for nutrient in range(0, num_nutrients):
            nutrients[nutrient] += config['food_data'][i][nutrient+2] * amount[i].solution_value()

        logger.info('%s = %f' % (config['food_data'][i][0], amount[i].solution_value()))

    logger.info(f'Total amount (g): {amount_total}')
    logger.info(f"Optimal cost per can of {config['weight_per_can']}g ($): {price_total*config['weight_per_can']}")

else:  # No optimal solution was found.
    if status == solver.FEASIBLE:
        print('A potentially suboptimal solution was found.')
    else:
        print('The solver could not solve the problem.')

2020-02-12 22:31:29.641 | INFO     | __main__:<module>:15 - chicken = 0.000000
2020-02-12 22:31:29.643 | INFO     | __main__:<module>:15 - beef = 0.600000
2020-02-12 22:31:29.646 | INFO     | __main__:<module>:15 - mutton = 0.000000
2020-02-12 22:31:29.648 | INFO     | __main__:<module>:15 - rice = 0.000000
2020-02-12 22:31:29.651 | INFO     | __main__:<module>:15 - wheat = 0.000000
2020-02-12 22:31:29.653 | INFO     | __main__:<module>:15 - gel = 0.400000
2020-02-12 22:31:29.654 | INFO     | __main__:<module>:17 - Total amount (g): 1.0
2020-02-12 22:31:29.655 | INFO     | __main__:<module>:18 - Optimal cost per can of 100g ($): 0.52
